In [2]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
from scipy import cluster
from random import randint
import time
df=pd.read_csv("yellow_tripdata_2015-07.csv").sample(10000,random_state=1)

#adopt ricky's implementation regards data preprocessing

In [3]:

df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,...,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,PickupCell,DropoffCell,Unnamed: 21
5533900,1,2015-07-16 12:17:09,2015-07-16 12:25:05,2,1.30,-73.991470,40.739048,1,N,-74.006851,...,7.0,0.0,0.5,1.00,0.00,0.3,8.80,26:70,24:73,NaN
10464409,1,2015-07-29 09:51:13,2015-07-29 10:14:47,1,4.10,-73.994011,40.751160,1,N,-74.013603,...,18.5,0.0,0.5,4.82,0.00,0.3,24.12,24:68,26:77,NaN
1196437,2,2015-07-04 15:29:20,2015-07-04 15:35:41,1,1.02,-73.984238,40.758743,1,N,-73.969475,...,6.0,0.0,0.5,0.00,0.00,0.3,6.80,25:66,27:64,NaN
1220618,1,2015-07-04 17:24:08,2015-07-04 17:30:28,1,1.20,-73.965233,40.774536,1,N,-73.981430,...,6.5,0.0,0.5,1.00,0.00,0.3,8.30,26:61,24:63,NaN
8184125,1,2015-07-23 09:17:11,2015-07-23 09:39:31,2,6.60,-74.002144,40.739681,3,N,-74.067192,...,39.5,0.0,0.0,20.00,11.75,0.3,71.55,NaN,NaN,NaN


In [4]:

#stay consistent with ricky's data preprocessing 




# Convert string type to datetime
df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])
# Total earninigs = tip_amount + total_amount
df['total_earnings'] = df["tip_amount"] + df['total_amount']
# Calculate trip_duration in seconds
df["trip_duration"] = (df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']).dt.total_seconds()
# Extract day of week and minute of day
df["day_of_week"] = df["tpep_pickup_datetime"].dt.weekday.astype(str)
df["minute_of_day"] = df["tpep_pickup_datetime"].dt.time
# Round down time to nearest minute
df['minute_of_day'] =  [i.replace(second=0) for i in df['minute_of_day']]
# Compute frequency and TER
df_ter = df.groupby(["PickupCell", "day_of_week", "minute_of_day"]) \
            .size() \
            .to_frame(name='frequency') \
            .join(df.groupby(["PickupCell", "day_of_week", "minute_of_day"]) \
                    .agg({"total_earnings":"mean", "trip_duration":"mean"}) \
                    .rename(columns={"trip_duration":"avg(td)", "total_earnings":"avg(te)"}) \
            ).reset_index()
df_ter['avg(ter)'] = df_ter['avg(te)']/df_ter['avg(td)']
df_ter.to_csv("df_score_table.csv", index=False)
df_ter['day_of_week'][0]

'4'

In [5]:
df_st = pd.read_csv("df_score_table.csv")
df_st.dtypes

PickupCell        object
day_of_week        int64
minute_of_day     object
frequency          int64
avg(te)          float64
avg(td)          float64
avg(ter)         float64
dtype: object

In [6]:
X = df_st.loc[:, ["PickupCell","day_of_week","minute_of_day"]]
y = df_st.loc[:,"frequency"].astype('float')

In [7]:
import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
ohe = OneHotEncoder(sparse=False) #Easier to read

In [8]:
df_st['nx'] = df_st.loc[:, ["PickupCell","day_of_week","minute_of_day"]].apply(
    lambda x: ','.join(x.dropna().astype(str)),
    axis=1
)


In [9]:
df_st['nx']=df_st['nx'].str.replace(",","")
df_st['nx']=df_st['nx'].str.replace(":","")
X2 = df_st.loc[:, ["nx"]].astype(int)
X2

,nx
0,21424005900
1,21453005400
2,21673115000
3,21675183600
4,21680113200
5,21681211500
6,21682085200
7,21682182900
8,21682203300
9,21683081200


In [10]:
res = sm.GLM(y, X2, family=sm.families.Poisson()).fit()

In [ ]:
train = ohe.fit_transform(X2).toarray()

train